# Monad

## Outline

* Incentive for Monad

* Definition of Monad type class

* Monads and "do" notation

* Monad examples

* Monad laws

In this lesson, we will learn about the Monad type classe and how you can use it.

## Incentive for Monad

We covered until now two cases where the Functor and Applicative type class functions can help us. 

If your input data is in a `Maybe` cotext and you have the following functions:
```haskell
f1 :: a -> b
f2 :: Maybe (a -> b) 
```
then you can use the `<$>` and `<*>` operators to apply them to your data of type `Maybe` and get data of type `Maybe` returned. 

This works for the `Maybe` type because it has an instance of Functor and Applicative. 

You can also define your custom type and make it an instance of Functor and Applicative.

To describe it in a more general way: 
- we use Functors to solve the problem of mapping a function over a value inside a context

- we use Applicative to solve the problem of what happens if the function is also inside the context.

But what if we have a value with a context `(m a)` and a funciton that takes a contextless value and returns a value with a context `f :: (a -> m b)`? 

Let's take an example for the Maybe type that provides the context for missing values:
```haskell
f3 :: a -> Maybe b
```

Below is an example with `Map` objects. We define book codes that map to book IDs. Then we define book prices that map to book codes.

We write a function that takes a book ID and returns a Maybe book prices. We use the `fromJust` and `isJust` functions from the **Data.Maybe** module.

In [ ]:
import Data.Maybe (fromJust, isJust)
import qualified Data.Map as Map

bookCodes :: Map.Map Int Int
bookCodes = Map.fromList $ zip [1,2,3] [2869, 7435, 1423]

bookPrices :: Map.Map Int String
bookPrices = Map.fromList $ zip [2869, 7435, 1423] ["15EUR","20USD","18JPY"]

getPriceFromID :: Int -> Maybe String
getPriceFromID n 
    | isJust id = Map.lookup (fromJust id) bookPrices
    | otherwise = Nothing
    where id = Map.lookup n bookCodes

print $ getPriceFromID 1

The Monad type class provides an existing solution for solving the problem above.

## Definition of the Monad type class

The definition of Monad type class is the following:
```haskell
class Applicative m => Monad m where
  (>>=) :: m a -> (a -> m b) -> m b
  (>>) :: m a -> m b -> m b
  return :: a -> m a
  {-# MINIMAL (>>=) #-}
```

We see that Applicative is a superclass of Monad. The minimal complete definition requires just the bind operator `>>=`. 

The bind operator takes a value inside a context and a function that takes such a value without the context and returns another type with the context.

What it does is that it extracts the value from the context applise the function to it and returns the result.

The sequncing operator `>>` takes two values in a context and returns the second one.

The `return` function just puts a value inside a context.

We notice that the `return` function does the same as the `pure` function from Applicative. In the source code it is defined as `return = pure`.

Ussually when creating an instance of monad for a type we define `return` and then when defining the Applicative type we say `pure = return`.

The Maybe type has also an instance of Monad type class. Let's see how we can use the bind operator to rewrite the previous code example. 

We will use the `flip` function that reverses the order of input parameters for a function.
```haskell
flip :: (a -> b -> c) -> b -> a -> c
```

In [ ]:
getPriceFromIDBind :: Int -> Maybe String
getPriceFromIDBind n = id >>= flip Map.lookup bookPrices
    where id = Map.lookup n bookIDs

print $ getPriceFromIDBind 1

We get the same result as before and the function is two lines shorter. 

The main reason why to use this operator is that it's easier to read the code because it's generic. The same symbol serves the same purpose for all Monads.

And some of Haskell existing type already have a Monad instance so you can use its functionality for free.

In the next chapter we will explain how the sequencing operator and the return function work.

## Monads and "do" notation

We notice that `return` looks the same as `pure` from the Applicative type class. In fact it does the same thing: puts a variavle into a context. 

The reason why they both exist is that the Monad type class predates the Applicative type class. 

When Applicative was developed `pure` was added to it because you can then express the `<$>` operator by using only `pure` and `<*>`. 

The `>>` operator takes two variables in a monad context and throws away the first. That makes sense when you want to chain commands together. 

Let's have a look of a function that runs in IO which has also an instance of Monad tpye class.

In [ ]:
chain :: String -> IO ()
chain msg = print "My message is:\n" >> print msg

chain "test"

We can now write a simple program that asks a user for his name and then prints back a greeting. We can use both the `>>` and the `>>=` operators.

In [ ]:
hello :: IO () 
hello = 
  print "What is your name?" >> 
  getLine >>= 
  (\name -> return ("Hello " ++ name)) >>= 
  print

hello

If we re-write the above program by using `do` notation Haskell translate it in the background to the above code. 

In other words the `do` notation is just a syntactic sugar (a nicer way of writing the same thing) for the code above.

In [ ]:
hello' :: IO () 
hello' = do 
  print "What is your name?" 
  name <- getLine 
  print ("Hello " ++ name) 

hello'

If you compare the code you see that the `<-` arrow is used to assign a variable that comes in a context to a variable without that context. 

In our case the `getLine` function returns an `IO String` and the variable `name` is just of type `String`. The statements are then chained together with the `>>` and `>>=` operators.

As you may remember you can also declare some variables inside a `do` block with a `let`. 

Below is the code that shows the `do` notation code and the de-sugarrated code that uses lambda functions and the Monad operators.

The functions `monadExample` and `monadExample'` basically do the same thing.

In [ ]:
monadExample :: IO Int 
monadExample = do 
  let a = return 1 :: IO Int 
  let func var = return (var + 1) :: IO Int 
  a >>= func

monadExample' :: IO Int 
monadExample' = 
  (\a -> 
    a >>= 
      (\var -> 
        return (var + 1) :: IO Int 
      ) 
  ) (return 1 :: IO Int)

main :: IO ()
main = do
    var1 <- monadExample
    var2 <- monadExample'
    print var1
    print var2

main

The `let` and `<-` statements are both used to assign variables but the diference is that the `<-` operator takes variables out of the context and stores them as a plain type. 

The declaration of variables with the `let` operator is translated by using lambda functions in the background.

## Monad examples

Until now we explained how the IO Monad works. You can use the same concept for other types that have an instance of Monad. Lets take for example a list.

In [ ]:
add1 :: Num a => [a] -> [a]
add1 myList = do
    element <- myList
    return $ element + 1

print $ add1 [1..3]

When you run the code above you see that even though you perform the (+ 1) operation on a signle element you get a list returned. 

You could generalize this function by using the Monad type constraint in the type signature and it would work for lists, maybe and other types that have an instance of Monad value.

In [ ]:
import qualified Data.Map as Map

add1Monad :: (Num a, Monad m) => m a -> m a
add1Monad var = do
    raw <- var
    return $ raw + 1

print $ add1Monad [1..3]
print $ add1Monad $ Just 1

Another example of Monads is **list comprehension** which is an elegant way to create a list by specifying the element conditions inside the list. Here is a example:

In [ ]:
import Control.Monad ( guard )

list1 = [0 .. 9 :: Int]

add1Even :: [Int] -> [Int]
add1Even myList = do
  n <- myList
  let k = n + 1
  guard(even k)
  return k
  
add1Even' :: [Int] -> [Int]
add1Even' myList = [k | n <- myList, let k = n + 1, even k] 

print $ add1Even list1
print $ add1Even' list1

The `guard` function is basically a filter that passes only values of `k` which give `True` fot the expression inside the brackets. 

The function `add1Even` is just a de-sugarated version of the function `add1Even'` and they work the same. 

In our last example lets create a custom type called `Writer` that is parameterized by the types `Int` and `String`. 
```haskell
data Writer a = Writer a String deriving Show
```
The string contains a message as for example the integer value that we stored in the `Int` type. 

We make it an instance of Monad for this type. For this we need to define the bind operator.

When we apply the bind perator to a Writer type its message and should be concatenated with the message of the Writer type that the function returns.

In [ ]:
import Control.Monad (ap, liftM)

data Writer a = Writer a String
    deriving Show

bindWriter :: Writer a -> (a -> Writer b) -> Writer b
bindWriter (Writer a xs) f =
  let
    Writer b ys = f a
  in
    Writer b $ xs ++ ys

instance Functor Writer where
    fmap = liftM

instance Applicative Writer where
    pure = return
    (<*>) = ap

instance Monad Writer where
    return a = Writer a ""
    (>>=) = bindWriter

In the above example we see that we have to also declare instances for Functor and Applicative. 

This is because Applicative is a superclass of Monad and Functor is a superclass of Applicative. 

We do this with the functions `ap` and `liftM` that we import from the `Control.Monad` module. 

We will explain those functions in detail in lesson 24. 

Then we write a function that takes in three `Writer` variables and summs the numbers together and cocatenates the messages.

We also create a `tell` function that just creates a Writer with the input message and an empty value.

In [ ]:
tell :: String -> Writer ()
tell msg = Writer () msg

tellAndSum :: Writer Int -> Writer Int -> Writer Int -> Writer Int
tellAndSum x y z = do
    tell "Sumed numbers: " 
    sumInts x y z

sumInts :: Monad m => m Int -> m Int -> m Int -> m Int
sumInts mx my mz = do
    k <- mx
    l <- my
    m <- mz
    let s = k + l + m
    return s

print $ tellAndSum (Writer 1 "1 ") (Writer 2 "2 ") (Writer 3 "3")

Also you might ask yourself how does this code work when it concatenates the messages? You have to look at the definition of the `>>` operator which is expressed with `>>=`. 

If you type `:i (>>)` you will see the comment `-- Defined in ‘GHC.Base’` which means it's defined in the source code of GHC [(1)](https://github.com/ghc/ghc/blob/master/libraries/base/GHC/Base.hs). 

As we said the minimal complete definition for Monad is only `>>=` so all other operators are expressed by using `>>=`. 

Here is the `>>` definition from the source code:
```haskell
(>>) :: forall a b. m a -> m b -> m b
m >> k = m >>= \_ -> k
{-# INLINE (>>) #-}
```

In our Writer example we defined `>>=` (which represents `bindWriter`) in such a way that the message from `Writer a` gets concatenated with the message from `Writer b`. 

So when using the `>>` operator which actually uses the `>>=` operator, this concatination of messages also happens.
```haskell
(Writer 1 "1 ") >> (Writer 2 "and 2") 
-- returns Writer 2 "1 and 2"
```

## More complex examples

As stated in the applicative lesson functions in general have also an instance of Monad type class. 

So you can re-write the fibonacci example using the bind operator.

In [ ]:
fibs = 0:1:(tail >>= zipWith (+)) fibs

print $ take 5 fibs

With the Monad operators it is possible also to express the operators for Functor and Applicative.

In [ ]:
fmapM :: Monad m => (a -> b) -> m a -> m b
fmapM func val = val >>= (\x -> return (func x))

appM :: Monad m => m (a -> b) -> m a -> m b
appM func val = func >>= (\f -> val >>= (\x -> return (f x)))

print $ fmapM (+1) $ Just 1
print $ appM (fmapM (+) $ Just 1) (Just 1)

## Monad laws

As for Functor and Applicative also the Monad type class has also its origins in mathematics and is defined with laws.

Monad has 3 laws. They are as follows:

- Identity (left):<br>`return a >>= f = f a`

- Identity (right):<br>`m >>= return = m`

- Associativity:<br>`(a >>= b) >>= c = a >>= (\x -> b x >>= c)`

The left identity law ensures that the bind operator `>>=` does nothing else as just take the input argument out of the context and apply the function to it.

The right identity law ensures that when using the return function it should not change anything except put the data back into context.

The associativity law ensures that the way we group the monadic actions doesn’t matter because `>>=` ensures that they’ll be evaluated in a consistent order.

Below is example code that proves these laws hold for the Maybe type which has an instance of Monad.

In [ ]:
-- Left identity law
((return 1 :: Maybe Int) >>= (\x -> return (x + 1) :: Maybe Int)) == (\x -> return (x + 1) :: Maybe Int) 1

-- Right identity law
((Just 1) >>= return) == Just 1

-- Associativity law
var1 :: Maybe Int
var1 = Just 1

add1, mult2 :: Int -> Maybe Int
add1 x = return (x + 1)
mult2 x = return (2 * x)

((var1 >>= add1) >>= mult2) == (var1 >>= (\x -> add1 x >>= mult2))

Let's look now at an example where we create a type and an monad instance for it that violates these laws.

In [ ]:
import Control.Monad (ap, liftM)

data NotOk a = NotOk a Bool deriving (Eq, Show)

bindNotOk :: NotOk a -> (a -> NotOk b) -> NotOk b
bindNotOk (NotOk a myBool) f =
  let
    NotOk b myBool' = f a
  in
    NotOk b $ not myBool

instance Functor NotOk where
    fmap = liftM

instance Applicative NotOk where
    pure = return
    (<*>) = ap

instance Monad NotOk where
  (>>=) = bindNotOk
  return a = NotOk a True

var1 :: NotOk Int
var1 = NotOk 1 True

add1, mult2 :: Int -> NotOk Int
add1 x = NotOk (x + 1) True
mult2 x = NotOk (2 * x) True

print $ ((return 1 :: NotOk Int) >>= add1) == add1 1
print $ (var1 >>= return) == var1
print $ ((var1 >>= add1) >>= mult2) == (var1 >>= (\x -> add1 x >>= mult2))

Same as for Functor and Applicative it is not obligatory to follow these laws in Haskell if you create a type and make an instance of Monad for it.

It is still good practice to follow them because:
- you can better reason about what your code is doing

- you can make use of other functions that work with the Monad type class

## Recap

In this lesson we've discussed:

- the motivation for introducing the Monad type 

- definition of the Monad type class

- the Monad operators and "do" notation

- examples that show how to use the Monad type class

- laws that apply to the Monad type class